## Tracer correlation factor for 2d dumbell diffusion on a {110} family plane in a BCC crystal - see Appendix B in our paper 

In [1]:
import sys
sys.path.append("../")
sys.path.append("../../")

In [2]:
import numpy as np
from Onsager_calc_db import *
import onsager.crystal as crystal
from states import *
from stars import *
from vector_stars import *
import pickle
from scipy.constants import physical_constants
kB = physical_constants['Boltzmann constant in eV/K'][0]
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker

/usr/local/lib/python3.6/dist-packages/onsager-1.3.3-py3.6.egg/onsager/cluster.py:835: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.


In [3]:
a2 = 1.0
a1 = np.sqrt(2)*a2

crys2d = crystal.Crystal(np.array([[a1, 0.], [0., a2]]),
                         [[np.array([0., 0.]), np.array([0.5, 0.5])]], ["A"])
print(crys2d)

#Lattice:
  a1 = [0.70710678 0.5       ]
  a2 = [-0.70710678  0.5       ]
#Basis:
  (A) 0.0 = [0. 0.]


In [4]:
o = np.array([0.2, 0.])
famp0 = [o.copy()]
family = [famp0]
pdbcontainer = dbStates(crys2d, 0, family)
mdbcontainer = mStates(crys2d, 0, family)

In [5]:
cutoff = 1.01*np.linalg.norm(crys2d.lattice[:, 0])
print(cutoff)
jset0, jset2 = pdbcontainer.jumpnetwork(cutoff, 0.01, 0.01), mdbcontainer.jumpnetwork(cutoff, 0.01, 0.01)

0.8746856578222831


In [6]:
len(jset2)

2

In [7]:
print(jset2[0][1][0])

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0]
	dumbbell : (i, or) index = 0, lattice vector = [0 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [-1  0]
	dumbbell : (i, or) index = 1, lattice vector = [-1  0]
Jumping from c1 = 1 to c2 = 1


In [8]:
# Modify jnet0
jnet0 = jset0[0]
jnet0_indexed = jset0[1]
# Let's try to sort the jumps according to closest distance
# except rotational jumps, we don't want them.
z = np.zeros(crys2d.dim)
indices = []

for jt, jlist in enumerate(jnet0):
    if np.allclose(jnet0_indexed[jt][0][1], z):
        continue
    indices.append(jt)
    
def sortkey(entry):
    jmp = jnet0[entry][0]
    or1 = pdbcontainer.iorlist[jmp.state1.iorind][1]
    or2 = pdbcontainer.iorlist[jmp.state2.iorind][1]
    dx = disp(pdbcontainer, jmp.state1, jmp.state2)
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3
ind_sort = sorted(indices, key=sortkey)
# ind_sort

In [9]:
jset0new = ([jnet0[ind_sort[0]]], [jnet0_indexed[ind_sort[0]]])

In [10]:
# Modify jnet2
jnet2 = jset2[0]
jnet2_indexed = jset2[1]
# Let's try to sort the jumps according to closest distance
# we don't want the rotational jumps as before.
z = np.zeros(crys2d.dim)
indices2 = []
for jt, jlist in enumerate(jnet2):
    if np.allclose(jnet2_indexed[jt][0][1], z):
        continue
    indices2.append(jt)

def sortkey2(entry):
    jmp = jnet2[entry][0]
    or1 = mdbcontainer.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer.iorlist[jmp.state2.db.iorind][1]
    dx = disp(mdbcontainer, jmp.state1, jmp.state2)
    # c1 and c2 are always +1 for mixed dumbbell jumps.
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3

ind_sort2 = sorted(indices2, key=sortkey2)
print(ind_sort2)

[1, 0, 2]


In [11]:
jset2new = ([jnet2[ind_sort2[0]]], [jnet2_indexed[ind_sort2[0]]])

In [12]:
start = time.time()
onsagercalculator = dumbbellMediated(pdbcontainer, mdbcontainer, jset0new, jset2new, cutoff,
                                     0.01, 0.01, 0.01, NGFmax=4, Nthermo=1)
print("onsager calculator initiation time = {}".format(time.time() - start))

initializing thermo
initializing kin
initializing NN
built shell 1: time - 0.0021457672119140625
built shell 2: time - 0.009932994842529297
grouped states by symmetry: 0.0235135555267334
built mixed dumbbell stars: 0.00015878677368164062
built jtags2: 6.270408630371094e-05
built mixed indexed star: 0.0004565715789794922
building star2symlist : 9.679794311523438e-05
building bare, mixed index dicts : 0.00017571449279785156
2NN Shell initialization time: 0.05163240432739258

generating thermodynamic shell
built shell 1: time - 0.0002942085266113281
grouped states by symmetry: 0.002771139144897461
built mixed dumbbell stars: 7.724761962890625e-05
built jtags2: 6.628036499023438e-05
built mixed indexed star: 0.00023937225341796875
building star2symlist : 2.8133392333984375e-05
building bare, mixed index dicts : 7.176399230957031e-05
thermodynamic shell generated: 0.005335330963134766
Total number of states in Thermodynamic Shell - 5, 2
generating kinetic shell
built shell 1: time - 0.00069

In [13]:
len(onsagercalculator.jnet43)

4

In [14]:
onsagercalculator.om1types

[0, 0, 0, 0]

In [15]:
jnet43 = onsagercalculator.jnet43
jnet43_indexed = onsagercalculator.jnet43_indexed
# Let's try to sort the jumps according to closest distance
# we don't want the rotational jumps as before.

z = np.zeros(crys2d.dim)
indices43 = []
for jt, jlist in enumerate(jnet43):
    if np.allclose(jnet43_indexed[jt][0][1], z):
        continue
    indices43.append(jt)    
# print(indices43)

def sortkey43(entry):
    jmp = jnet43[entry][0] # This is an omega4 jump
    if not jmp.c2 == -1:
        print(c2)
    or1 = pdbcontainer.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer.iorlist[jmp.state2.db.iorind][1]
    dx = disp4(pdbcontainer, mdbcontainer, jmp.state1, jmp.state2)
    # remember that c2 is -1 for an omega4 jump
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx - or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(jmp.c2*or2/2.)
    return dx1+dx2+dx3

ind_sort43 = sorted(indices43, key=sortkey43)
print(ind_sort43)

[2, 0, 3, 1]


In [16]:
print(len(jnet43[ind_sort43[0]]))

8


In [17]:
# Check the omega43 jumps
for jump in jnet43[ind_sort43[0]][0::2]:
    print(jump)
    print()

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0]
	dumbbell : (i, or) index = 0, lattice vector = [ 0 -1]
Final state:
	Solute loctation :basis index = 0, lattice vector = [0 0]
	dumbbell : (i, or) index = 0, lattice vector = [0 0]
Jumping from c1 = 1 to c2 = -1

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0]
	dumbbell : (i, or) index = 0, lattice vector = [0 1]
Final state:
	Solute loctation :basis index = 0, lattice vector = [0 0]
	dumbbell : (i, or) index = 1, lattice vector = [0 0]
Jumping from c1 = -1 to c2 = -1

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0]
	dumbbell : (i, or) index = 0, lattice vector = [1 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [0 0]
	dumbbell : (i, or) index = 0, lattice vector = [0 0]
Jumping from c1 = 1 to c2 = -1

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0]
	dumbbell : (i, o

In [18]:
mdbcontainer.iorlist

[(0, array([-0.2,  0. ])), (0, array([0.2, 0. ]))]

In [19]:
pdbcontainer.iorlist

[(0, array([-0.2,  0. ]))]

In [20]:
# regenerate expansions with new 4,3 jump list
onsagercalculator.regenerate43([ind_sort43[0]])

In [21]:
# 1.  First get the rates and thermodynamic data
# All the energies of the "mixed" and pure dumbbells will be the same,
# All the jump rates will be the same
    # Since we have only one type each of omega0, omega2 and omega43 jumps, set their rates to zero.
    # All omega1 rates will be the same as the above rate.
# The "solute" energies will be zero since we are dealing with a chemically identical tracer.
# All interaction energies will be zero.

# 1a. Energies and pre-factors
kT = 1

predb0, enedb0 = np.ones(len(onsagercalculator.vkinetic.starset.pdbcontainer.symorlist)), \
                 np.ones(len(onsagercalculator.vkinetic.starset.pdbcontainer.symorlist))

preS, eneS = np.ones(
    len(onsagercalculator.vkinetic.starset.crys.sitelist(onsagercalculator.vkinetic.starset.chem))), \
             np.zeros(len(onsagercalculator.vkinetic.starset.crys.sitelist(
                 onsagercalculator.vkinetic.starset.chem)))

# These are the interaction or the excess energies and pre-factors for solutes and dumbbells.
# The energies will all be zero.
preSdb, eneSdb = np.ones(onsagercalculator.thermo.mixedstartindex), \
                 np.zeros(onsagercalculator.thermo.mixedstartindex)

predb2, enedb2 = predb0.copy(), enedb0.copy()

preT0, eneT0 = np.ones(len(onsagercalculator.vkinetic.starset.jnet0)), np.ones(len(onsagercalculator.jnet0))
preT2, eneT2 = preT0.copy(), eneT0.copy()
preT1, eneT1 = np.ones(len(onsagercalculator.jnet1)), np.array([eneT0[onsagercalculator.om1types[jt]] for jt in
                                                                range(len(onsagercalculator.jnet1))])

preT43, eneT43 = np.ones(len(onsagercalculator.jnet43)), eneT0.copy()

In [22]:
# 1b. Now get the beta*free energy values.
bFdb0, bFdb2, bFS, bFSdb, bFT0, bFT1, bFT2, bFT3, bFT4 = \
    onsagercalculator.preene2betafree(kT, predb0, enedb0, preS, eneS, preSdb, eneSdb, predb2, enedb2,
                                           preT0, eneT0, preT2, eneT2, preT1, eneT1, preT43, eneT43)

In [23]:
len(onsagercalculator.jnet1)

4

In [24]:
# get the probabilities and other data from L_ij
L0bb,(L_uc_aa,L_c_aa), (L_uc_bb,L_c_bb), (L_uc_ab,L_c_ab)=\
onsagercalculator.L_ij(bFdb0, bFT0, bFdb2, bFT2, bFS, bFSdb, bFT1, bFT3, bFT4)

In [25]:
L_aa = L_uc_aa + L_c_aa
L_ab = L_uc_ab + L_c_ab

In [26]:
L_ab

array([[1. , 0. ],
       [0. , 0.5]])

In [27]:
L_aa

array([[0.241453, 0.      ],
       [0.      , 0.5     ]])

In [28]:
np.trace(L_aa)/np.trace(L_ab)

0.4943020016465871

In [29]:
np.trace(L_aa)/np.trace(L_uc_aa)

0.4943020016465871